# Daily Feature Pipeline
* Retrieve todays data for flights and google trends
* Add these new data to the Feature Store 

## OpenSky Recent Data 
* Use the OpenSky api to retrieve the most recent flight landing data, to update our feature group

In [5]:
import pandas as pd 
import os
import datetime
import requests 
import hopsworks

In [6]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

2025-12-31 12:47:09,232 INFO: Initializing external client
2025-12-31 12:47:09,234 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-12-31 12:47:11,036 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296539


In [21]:
CLIENT_ID = secrets.get_secret("OPENSKY_CLIENT_ID").value
CLIENT_SECRET = secrets.get_secret("OPENSKY_CLIENT_SECRET").value
ICAO = "ESSA"

def get_access_token():
    auth_url = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }
    response = requests.post(auth_url, data=data)
    response.raise_for_status()
    return response.json().get("access_token")

def fetch_yesterday_data(token):
    # Calculate yesterday's window
    yesterday = datetime.date.today() - datetime.timedelta(days=1)
    start_ts = int(datetime.datetime.combine(yesterday, datetime.time.min).timestamp())
    end_ts = int(datetime.datetime.combine(yesterday, datetime.time.max).timestamp())

    api_url = f"https://opensky-network.org/api/flights/arrival?airport={ICAO}&begin={start_ts}&end={end_ts}"
    headers = {"Authorization": f"Bearer {token}"}
    
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        flights = response.json()
        return yesterday, len(flights)
    else:
        print(f"API Error: {response.status_code}")
        return yesterday, 0

token = get_access_token()
flight_date, total_landings = fetch_yesterday_data(token)

print(f"Arlanda had {total_landings} landings on date: {flight_date}")

Arlanda had 244 landings on date: 2025-12-29


## Uploading new data to the Feature Store 

**New Flight Data**

In [23]:
new_flight_data = pd.DataFrame({
    "date": [flight_date],
    "total_landings": [total_landings],
})

In [7]:
# Retrieve feature group
flight_data_fg = fs.get_feature_group(
    name='flight_data_arlanda',
    version=1,
)

In [25]:
# insert new data
flight_data_fg.insert(new_flight_data, wait = True)

Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time


Launching job: flight_data_arlanda_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1296539/jobs/named/flight_data_arlanda_1_offline_fg_materialization/executions
2025-12-30 13:15:15,998 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-12-30 13:15:19,206 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-12-30 13:15:25,595 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-12-30 13:17:05,219 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-12-30 13:17:05,387 INFO: Waiting for log aggregation to finish.
2025-12-30 13:17:14,118 INFO: Execution finished successfully.


(Job('flight_data_arlanda_1_offline_fg_materialization', 'SPARK'), None)

## Google Trends Recent Data
* Download the most recent google search data for the search terms

In [8]:
# Google Trends
from pytrends.request import TrendReq

In [12]:
# Search terms used as predictors
KEYWORDS = [
    "vikings",
    "fika",
    "stockholm",
    "ikea",
    "abba"
]

# Country code for Sweden
COUNTRY = "SE"

# Get Yesterdays Date
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

In [10]:
def fetch_google_trends(keywords, start_date, end_date):
    """
    Fetch Google Trends data for given keywords and date range.
    Returns time-series data indexed by date.
    The resolution (daily / weekly / monthly) is decided by Google Trends
    based on the length of the date range.
    """
    pytrends = TrendReq(hl="en-US", tz=360)

    timeframe = f"{start_date} {end_date}"

    pytrends.build_payload(
        kw_list=keywords,
        timeframe=timeframe,
        geo=COUNTRY
    )

    df = pytrends.interest_over_time()
    return df

In [16]:
raw_data = fetch_google_trends(KEYWORDS, yesterday, yesterday)
raw_data = raw_data.drop(columns=["isPartial"])

yeserday_data = (
    raw_data
    .resample("D")
    .ffill()
    .reset_index()
)

yeserday_data["date"] = pd.to_datetime(yeserday_data["date"]).dt.date
yeserday_data


TooManyRequestsError: The request failed: Google returned a response with code 429

In [17]:
# Retrieve feature group
google_data_fg = fs.get_feature_group(
    name='google_trends_daily',
    version=1,
)



In [18]:
feature_group.insert(
    yeserday_data,
    write_options={"wait_for_job": True}
)

NameError: name 'feature_group' is not defined